In [1]:
import pandas as pd


In [2]:
books = pd.read_csv('books.csv')

In [3]:
books.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [5]:
print(books.columns.tolist())


['id', 'book_id', 'best_book_id', 'work_id', 'books_count', 'isbn', 'isbn13', 'authors', 'original_publication_year', 'original_title', 'title', 'language_code', 'average_rating', 'ratings_count', 'work_ratings_count', 'work_text_reviews_count', 'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url', 'small_image_url']


In [6]:
books['text_for_analysis'] = (
    books['title'].fillna('') + ' by ' +
    books['authors'].fillna('') +
    '. Rated ' + books['average_rating'].astype(str) + ' by users.'
)


In [7]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

books['sentiment_score'] = books['text_for_analysis'].apply(lambda x: sia.polarity_scores(x)['compound'])

# Label the sentiment
def label_sentiment(score):
    if score >= 0.5:
        return 'positive'
    elif score <= -0.5:
        return 'negative'
    else:
        return 'neutral'

books['sentiment_label'] = books['sentiment_score'].apply(label_sentiment)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/divyashree/nltk_data...


In [8]:
books[['book_id', 'title', 'authors', 'average_rating', 'sentiment_score', 'sentiment_label']].to_csv('sentiment_books.csv', index=False)


In [10]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

# Check on 5 examples
for i in range(5):
    text = books.iloc[i]['text_for_analysis']
    print("VADER:", books.iloc[i]['sentiment_label'])
    print("Transformers:", classifier(text)[0])
    print("---")


/Users/divyashree/PycharmProjects/Sentiment_book_recommender/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/divyashree/PycharmProjects/Sentiment_book_recommender/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


VADER: neutral
Transformers: {'label': 'POSITIVE', 'score': 0.9823432564735413}
---
VADER: neutral
Transformers: {'label': 'POSITIVE', 'score': 0.9928637146949768}
---
VADER: neutral
Transformers: {'label': 'POSITIVE', 'score': 0.9937421679496765}
---
VADER: negative
Transformers: {'label': 'POSITIVE', 'score': 0.9630486369132996}
---
VADER: positive
Transformers: {'label': 'POSITIVE', 'score': 0.9970598816871643}
---


In [13]:
!pip3 install transformers torch



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
from transformers import pipeline

# Load the sentiment analysis pipeline
classifier = pipeline("sentiment-analysis")

# Example: classify book title text
text = "Harry Potter and the Prisoner of Azkaban by J.K. Rowling. Rated 4.53 by users."
result = classifier(text)[0]
print(f"Label: {result['label']}, Score: {result['score']:.4f}")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


Label: POSITIVE, Score: 0.9818


In [15]:
def get_sentiment_label(text):
    try:
        result = classifier(text)[0]
        return result['label'].lower()  # 'positive', 'negative'
    except:
        return 'neutral'

# Create synthetic description as before
books['text_for_analysis'] = (
    books['title'].fillna('') + ' by ' +
    books['authors'].fillna('') + '. Rated ' + books['average_rating'].astype(str)
)

# Apply transformer sentiment
books['transformer_sentiment'] = books['text_for_analysis'].apply(get_sentiment_label)

# Save updated CSV
books[['book_id', 'title', 'authors', 'average_rating', 'transformer_sentiment']].to_csv('transformer_sentiment_books.csv', index=False)


In [16]:
!pip install transformers torch --quiet
!pip install emoji --quiet


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


Device set to use mps:0


In [31]:
import pandas as pd

# Load the dataset
df = pd.read_csv("books.csv")


In [22]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [23]:
label_map = {
    "LABEL_0": "negative",
    "LABEL_1": "neutral",
    "LABEL_2": "positive"
}

In [24]:
def get_sentiment(text):
    try:
        result = classifier(text[:512])[0]
        return label_map.get(result['label'], "unknown")
    except:
        return "error"

In [25]:
df["sentiment"] = df["title"].apply(get_sentiment)

In [29]:

import matplotlib
matplotlib.use('Agg')  # <-- Add this
import matplotlib.pyplot as plt


sentiment_counts = df['sentiment'].value_counts()

# Pie Chart
plt.figure(figsize=(5,5))
plt.pie(sentiment_counts, labels=sentiment_counts.index, autopct='%1.1f%%', startangle=140)
plt.title("Sentiment Distribution of Book Titles")
plt.axis('equal')
plt.show()

# OR Bar Chart
sentiment_counts.plot(kind='bar', color='skyblue')
plt.title("Sentiment Counts")
plt.xlabel("Sentiment")
plt.ylabel("Number of Books")
plt.xticks(rotation=0)
plt.show()


/var/folders/m1/9p_lvj592mgbcgbqlnjhlzhm0000gn/T/ipykernel_18077/3680994412.py:13: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/m1/9p_lvj592mgbcgbqlnjhlzhm0000gn/T/ipykernel_18077/3680994412.py:21: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [30]:
df.to_csv("books_with_sentiment.csv", index=False)
print("✅ Saved with sentiment column!")


✅ Saved with sentiment column!
